<a href="https://colab.research.google.com/github/Summit2/ML_education/blob/main/Comparison_of_GD_methods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np

import seaborn as sns
sns.set(style="darkgrid", font_scale=1.4)
from matplotlib import pyplot as plt

from sklearn.model_selection import train_test_split

import torch
from torch import nn
from torch.nn import functional as F

from torch.utils.data import TensorDataset, DataLoader

from torchvision.datasets import MNIST
from torchvision import transforms


In [5]:
data_transforms = transforms.Compose(
    [
     transforms.ToTensor(),
     transforms.Normalize((0.5),(0.5))

    ]

)

In [9]:
root = './'
train_dataset = MNIST(root, transform=data_transforms, download = True)
val_dataset = MNIST(root, transform=data_transforms,download = True)

train_dataloader = DataLoader(train_dataset, batch_size = 128)
val_dataloader =  DataLoader(val_dataset, batch_size = 128)

loaders = {'train' : train_dataloader,
           'valid' : val_dataloader }

In [11]:
# aka LeNet
LeNet = nn.Sequential(
    nn.Conv2d(1,6,3),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Conv2d(6,16,3),
    nn.ReLU(),
    nn.Flatten(),
    nn.Linear(400,120),
    nn.ReLU(),
    nn.Linear(120,84),
    nn.ReLU(),
    nn.Linear(84,10)

)


In [12]:
  device = 'cuda' if torch.cuda.is_available() else 'cpu'
  device

'cpu'

In [13]:
def train_loop(optimizer):
  model = LeNet()
  criterion = torch.nn.CrossEntropyLoss()
  optimizer = optimizer

  max_epochs = 10
  accuracy = {"train": [], "valid": []}